# This File to load daily data (5 min) into MongoDB

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime

In [2]:
def downloader_5(stock):
    data=getdata(stock,'Market5')
    start=data.tail(1).index[0]
    exchange=['.NS','.BO']
    try:
       # df_ns=yf.download(stock+exchange[0]).reset_index()
        ns=yf.download(stock+exchange[0],start=start,interval='5m')
        df_ns = ns.iloc[1: , :].copy()
        del ns    
    except:
        print(f'{stock+exchange[0]} not found')
            
    try:
        bo=yf.download(stock+exchange[1],start=start,interval='5m')
        df_bo = bo.iloc[1: , :].copy()
        del bo
        #df_bo=yf.download(stock+exchange[1]).reset_index()
    except:
        print(f'{stock+exchange[1]} not found')
    if not df_ns.empty:
        if not  df_bo.empty:
#             df_bo['Date']=pd.to_datetime(df_bo['Date'])
#             df_bo=df_bo.set_index('Date')
#             df_ns['Date']=pd.to_datetime(df_ns['Date'])
#             df_ns=df_ns.set_index('Date')
            if df_bo.shape[0]>1 & df_ns.shape[0]>1:
                df_ns.dropna(axis=0,how='any',inplace=True)
                df_bo.dropna(axis=0,how='any',inplace=True)
                df_ns['Volume']=df_ns['Volume'].astype(int)
                df_bo['Volume']=df_bo['Volume'].astype(int)
                df=pd.merge(df_ns,df_bo,on='Datetime',how='left')
                df=df.fillna(0)
                df['Volume']=df['Volume_x']+df['Volume_y']
                df=df[['Open_x','High_x','Low_x','Adj Close_x','Volume']]
                df=df.reset_index()
                df=df.rename(columns={'Open_x':'Open','High_x':'High','Low_x':'Low','Adj Close_x':'Close'})
                
                if df.iloc[-1: , -1:].values[0,0]==0:
                    df.drop(df.tail(1).index,inplace=True)
                return df
            elif df_ns.shape[0]>1:
                df_ns.dropna(axis=0,how='any',inplace=True)
                df_ns['Volume']=df_ns['Volume'].astype(int)
                df_ns=df_ns[['Open','High','Low','Adj Close','Volume']]
                df_ns=df_ns.reset_index()
                df_ns=df_ns.rename(columns={'Adj Close':'Close'})
                
                if df_ns.iloc[-1: , -1:].values[0,0]==0:
                    df_ns.drop(df_ns.tail(1).index,inplace=True)
                return df_ns
            
            else:
                df=pd.DataFrame()
                return df
            
        elif df_ns.shape[0]>1 :
            df_ns.dropna(axis=0,how='any',inplace=True)
            df_ns['Volume']=df_ns['Volume'].astype(int)
            df_ns=df_ns[['Open','High','Low','Adj Close','Volume']]
            df_ns=df_ns.reset_index()
            df_ns=df_ns.rename(columns={'Adj Close':'Close'})
            if df_ns.iloc[-1: , -1:].values[0,0]==0:
                df_ns.drop(df_ns.tail(1).index,inplace=True)
            return df_ns
    else:
        df=pd.DataFrame()
        return df

In [3]:
#This method read data from Mongo DB
def getdata(stock,db):
    stock=str(stock)
    mongo_client=MongoClient()
    db=mongo_client[db]
    data=db[stock].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    ###Covert to UTC to get the correct Values
    df['Datetime']=pd.to_datetime(df['Datetime']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
    #df['Datetime']=pd.to_datetime(df['Datetime'])
    df=df.set_index('Datetime')
    return df

In [4]:
# This Method load data into DB.
def to_db_5(stocklist):
    m_client=MongoClient()
    db=m_client["Market5"]
    print(f'5 min data Loading Starting')
    for stock in stocklist:
        print(f'{stock} 5 min data loading')
        stockdf=downloader_5(stock)
        try:
            if not stockdf.empty:
                data=stockdf.to_dict(orient='records')
                db[stock].insert_many(data)
                print (f'Data Load completed for stock {stock}')
            else:
                print(f'{stock} Nothing to load')
            
        except:
            print (f'Data Load failed for stock {stock}')
            pass
    print(f'5 min data Loading Ended')
    m_client.close()
    

In [5]:
#KSCL
#testdf=downloader_5('KSCL')

In [6]:
# To Fetch Stocks from DB
def get_stocklist(tbl):
    mongo_client=MongoClient()
    db=mongo_client["NIFTY500"]
    data=db[tbl].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    return df

In [7]:
df=get_stocklist('NIFTY500')
stocks=df.Symbol.to_list()

In [8]:
# Get the index of stock
#print(stocks.index('KSCL'))

In [9]:
#create new list of stocks starting from index you choose.
#stocks=stocks[277:]

In [10]:
#stocks

In [11]:
#provide list od stocks you want to downlad and load into mongo db
#stocks=['AAVAS']
#Calling load to db method 

to_db_5(stocks)



5 min data Loading Starting
3MINDIA 5 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data Load completed for stock 3MINDIA
ABB 5 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data Load completed for stock ABB
POWERINDIA 5 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data Load completed for stock POWERINDIA
ACC 5 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data Load completed for stock ACC
AIAENG 5 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data L